# Pangea AI Security Tools

Pangea's tools for LangChain provide AI security features to protect your applications and data. Using these tools you can:

- Defend against prompt injection attacks.
- Prevent the exposure of sensitive information, including:
  - Personally Identifiable Information (PII)
  - Protected Health Information (PHI)
  - Financial data
  - Secrets
  - Intellectual property
  - Profanity
- Remove malicious content from inputs and outputs, such as IP addresses, domains, and URLs.
- Monitor user inputs and model responses to support threat analysis, auditing, and compliance efforts.

## Prerequisites

### OpenAI API key

The examples below use OpenAI models. To run them, get your [OpenAI API key](https://platform.openai.com/api-keys) and export it as an environment variable:

- `OPENAI_API_KEY`

### Pangea project

Sign up for a free [Pangea account](https://pangea.cloud/signup) to access the security services required for these tools.

After signing up, click **Skip** on the **Get started with a common service** screen. This will take you to the Pangea User Console, where you can enable the specific services needed.

For details about Pangea services and their features, visit the Pangea website:
- [AI Guard](https://pangea.cloud/services/ai-guard/)
- [Redact](https://pangea.cloud/services/redact/)
- [Domain Intel](https://pangea.cloud/services/domain-intel/reputation/)
- [IP Intel](https://pangea.cloud/services/ip-intel/reputation/)
- [URL Intel](https://pangea.cloud/services/url-intel/)
- [Secure Audit Log](https://pangea.cloud/services/secure-audit-log/)

## Installation

In [ ]:
%pip install -qU langchain-pangea

## Tools

You can run Pangea tools using agents or invoke them as a Runnable within chains.

### AI Guard

#### Enable the AI Guard service

1. Open your [Pangea User Console](https://console.pangea.cloud).  
2. Click **AI Guard** in the left-hand sidebar and follow the prompts, accepting all defaults.  
3. When finished, click **Done** and then **Finish**. The enabled service will be marked with a green dot.  
4. On the service **Overview** page, capture the **Default Token** and **Domain** values by clicking their respective tiles. Export these values as environment variables:  
    - `PANGEA_DOMAIN`
    - `PANGEA_AI_GUARD_TOKEN`  

For more information on setting up the service and its usage, see the [AI Guard documentation](https://pangea.cloud/docs/ai-guard/).

#### Set up the environment

In [48]:
import os
from dotenv import load_dotenv
from pydantic import SecretStr

load_dotenv()

openai_api_key = SecretStr(os.getenv("OPENAI_API_KEY"))
pangea_domain = os.getenv("PANGEA_DOMAIN")
pangea_ai_guard_token = SecretStr(os.getenv("PANGEA_AI_GUARD_TOKEN"))

#### Define the model

In [49]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key.get_secret_value(), temperature=0)

#### Use the AI Guard tool with an agent

The following example demonstrates how the [LLM Response](https://pangea.cloud/docs/ai-guard/recipes#llm-response) (`pangea_llm_response_guard`) recipe can prevent sensitive or high-risk information from being returned to the user. Using this recipe you can:

- Defang malicious links (e.g., IPs, URLs, domains).  
- Redact specific personally identifiable information (PII) and secrets in the prompt, based on the rules defined in the recipe.

Recipes can be customized by adding, removing, or modifying rules. You can also discover, create, and configure additional recipes in your [Pangea User Console](https://console.pangea.cloud/service/ai-guard/recipes).

In [50]:
from langchain_pangea import PangeaAIGuard
from pangea import PangeaConfig

pangea_config = PangeaConfig(domain=pangea_domain)
pangea_ai_guard_tool = PangeaAIGuard(token=pangea_ai_guard_token, config=pangea_config, recipe="pangea_llm_response_guard")

In the example data, safe IP addresses are mixed with those listed on the [IPsum Threat Intelligence Feed](https://github.com/stamparm/ipsum). The AI Guard tool defangs IP addresses identified as dangerous, reducing the risk of users inadvertently using them.

The pre-built agent is instructed via a system message to apply the service recipe to the final result. Alternatively, you can create your agent and implement a more deterministic approach to ensure the service thoroughly sanitizes the LLM's response.

In [51]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

@tool
def search_tool(data):
    """Call to perform search"""

    return """
    47.84.32.175
    37.44.238.68
    47.84.73.221
    47.236.252.254
    34.201.186.27
    52.89.173.88
    """

tools = [search_tool, pangea_ai_guard_tool]

query = """
Hi, I am Bond, James Bond. I monitor IPs found in MI6 network traffic.
Please find me the most recent ones, you copy?
"""

system_message="Always use AI Guard before your final response to keep it safe for the user."

langgraph_agent_executor = create_react_agent(model, tools, state_modifier=system_message)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

The most recent IPs found in MI6 network traffic are: 47[.]84[.]32[.]175, 37[.]44[.]238[.]68, 47[.]84[.]73[.]221, 47[.]236[.]252[.]254, 34.201.186.27, 52.89.173.88.


#### Use AI Guard as a Runnable in chains

The following example demonstrates how the [LLM Prompt Pre-Send](https://pangea.cloud/docs/ai-guard/recipes#llm-prompt-pre-send) (`pangea_llm_prompt_guard`) recipe can be applied to prevent sensitive or high-risk information from being submitted to a public LLM, such as ChatGPT. Using this recipe you can:

- Defang malicious links (e.g., IPs, URLs, domains).
- Redact specific personally identifiable information (PII) and secrets in the prompt, based on the rules defined in the recipe.

Recipes can be customized by adding, removing, or modifying rules. You can also discover, create, and configure additional recipes in your [Pangea User Console](https://console.pangea.cloud/service/ai-guard/recipes).

In [52]:
from langchain_pangea import PangeaAIGuard
from pangea import PangeaConfig

pangea_config = PangeaConfig(domain=pangea_domain)
pangea_ai_guard_tool = PangeaAIGuard(token=pangea_ai_guard_token, config=pangea_config, recipe="pangea_llm_prompt_guard")

The user prompt includes some personally identifiable information.

The chain invokes the AI Guard tool before submitting the user prompt to the LLM.

In [53]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([("human", "{input}")])

query = """
Hi, I am Bond, James Bond. I am looking for a job. Please write me a super short resume.

I am skilled in international espionage, covert operations, and seduction.

Include a contact header:
Email: j.bond@mi6.co.uk
Phone: +44 20 0700 7007
Address: Universal Exports, 85 Albert Embankment, London, United Kingdom
"""

from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | pangea_ai_guard_tool
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

Sure! Here’s a concise resume for you:

---

**[Your Name]**  
Email: <EMAIL_ADDRESS>  
Phone: +44 20 0700 7007  
Address: Universal Exports, 85 *****************, London, United Kingdom  

---

**Objective**  
Dynamic and skilled professional seeking a challenging position in international espionage and covert operations.

---

**Skills**  
- Expertise in international espionage  
- Proficient in covert operations  
- Exceptional skills in seduction and interpersonal communication  

---

**Experience**  
- Conducted high-stakes intelligence operations in various global locations.  
- Developed and maintained covert relationships to gather critical information.  
- Successfully executed missions requiring discretion and strategic planning.  

---

**Education**  
- [Your Degree] in [Your Field]  
- [Your University], [Year of Graduation]  

---

**References**  
Available upon request.

--- 

Feel free to customize any sections as needed!


Note that not all personally identifiable information has been replaced or masked by the AI Guard tool. To apply stricter redaction rules, update the service recipes in your [Pangea User Console](https://console.pangea.cloud/service/ai-guard/recipes).

The same chain, without the AI Guard protection, will submit sensitive information to the LLM, providing it with personal context.

In [54]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

**James Bond**  
Email: j.bond@mi6.co.uk  
Phone: +44 20 0700 7007  
Address: Universal Exports, 85 Albert Embankment, London, United Kingdom  

---

**Objective**  
Dynamic and resourceful professional seeking a challenging position in international security and intelligence.

**Skills**  
- **International Espionage:** Extensive experience in gathering intelligence and conducting undercover operations across various global locations.  
- **Covert Operations:** Proven track record in executing high-stakes missions with precision and discretion.  
- **Seduction & Negotiation:** Exceptional interpersonal skills with a talent for persuasion and building rapport in diverse environments.  

**Experience**  
- **Secret Agent, MI6**  
  - Conducted numerous successful missions, neutralizing threats to national security.  
  - Collaborated with international agencies to gather intelligence and thwart criminal organizations.  
  - Utilized advanced technology and gadgets to enhance operational

#### Use AI Guard as a standalone tool

You can also call the AI Guard tool directly as needed.

In [95]:
print(pangea_ai_guard_tool.run("Spam me at example@example.com"))
print(pangea_ai_guard_tool.invoke("Take my SSN: 234-56-7890"))

Spam me at <EMAIL_ADDRESS>
Take my SSN: ***********


### Redact Guard

#### Enable the Redact service

1. Open your [Pangea User Console](https://console.pangea.cloud).  
2. Click **Redact** in the left-hand sidebar and follow the prompts, accepting all defaults.  
3. When finished, click **Done** and then **Finish**. The enabled service will be marked with a green dot.
4. On the service **Overview** page, capture the **Default Token**, **Config ID**, and **Domain** values by clicking their respective tiles. Save these values in the appropriate environment variables:
    - `PANGEA_DOMAIN`
    - `PANGEA_REDACT_TOKEN`
    - `PANGEA_REDACT_CONFIG_ID`

For more information on setting up the service and its usage, see the [Redact documentation](https://pangea.cloud/docs/redact/).

#### Set up the environment

In [56]:
import os
from dotenv import load_dotenv
from pydantic import SecretStr

load_dotenv()

openai_api_key = SecretStr(os.getenv("OPENAI_API_KEY"))
pangea_domain = os.getenv("PANGEA_DOMAIN")
pangea_redact_token = SecretStr(os.getenv("PANGEA_REDACT_TOKEN"))
pangea_redact_config_id = SecretStr(os.getenv("PANGEA_REDACT_CONFIG_ID"))

#### Define the model

In [57]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key.get_secret_value(), temperature=0)

#### Instantiate Redact Guard

The Redact service offers rules to detect, replace, mask, hash, or encrypt sensitive information in content sent to the AI app or returned to the user. You can customize these rules and add new ones in your [Pangea User Console](https://console.pangea.cloud/service/redact/rulesets). By default, three rules are enabled:

- Replace IP addresses.  
- Replace email addresses.  
- Replace US Social Security Numbers (SSN).  

In [58]:
from langchain_pangea import PangeaRedactGuard
from pangea import PangeaConfig

pangea_config = PangeaConfig(domain=pangea_domain, config_id=pangea_redact_config_id)
pangea_redact_guard_tool = PangeaRedactGuard(token=pangea_redact_token, config=pangea_config)

#### Define the context data and the user query

In this example, we will emulate a helpful HR assistant trained to return employee records.

In [59]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.tools import tool

@tool
def search_tool(data):
    """Call to perform HR record search"""

    return """
    Name: Jason Bourne
    Title: Rogue Operative
    Department: Former CIA Black Ops

    Email: j.bourne@unknown.gov
    Social Security Numbers:
    - 234-56-7890
    - 345-67-8901
    - 456-78-9012

    Hobbies:
    - Traveling
    - Using books and rolled-up newspapers as weapons
    """

query = """
Hi, I am Jason Bourne. What do you have on me?
"""

#### Use Redact Guard as an agent tool

In the following example, Redact Guard removes sensitive information from the data returned by the search tool.

In [60]:
from langgraph.prebuilt import create_react_agent

tools = [search_tool, pangea_redact_guard_tool]

langgraph_agent_executor = create_react_agent(model, tools)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

Here is the information I found on you, Jason Bourne:

- **Title:** Rogue Operative
- **Department:** Former CIA Black Ops
- **Email:** <EMAIL_ADDRESS>
- **Social Security Numbers:** 
  - <US_SSN>
  - <US_SSN>
  - <US_SSN>
- **Hobbies:**
  - Traveling
  - Using books and rolled-up newspapers as weapons

If you need more specific information or have any other questions, feel free to ask!


Note that not all personally identifiable information has been replaced by the Redact Guard tool. To apply stricter redaction rules, update the service recipes in your [Pangea User Console](https://console.pangea.cloud/service/ai-guard/recipes).

Without Redact Guard protection, sensitive information can be exposed to both the LLM and the user.

In [61]:
tools = [search_tool]

langgraph_agent_executor = create_react_agent(model, tools)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

Here is the information I found on you, Jason Bourne:

- **Title:** Rogue Operative
- **Department:** Former CIA Black Ops
- **Email:** j.bourne@unknown.gov
- **Social Security Numbers:**
  - 234-56-7890
  - 345-67-8901
  - 456-78-9012
- **Hobbies:**
  - Traveling
  - Using books and rolled-up newspapers as weapons

If you need more specific information or have any other questions, feel free to ask!


#### Use Redact Guard in a chain

In the following example, Redact Guard removes sensitive information from the additional context added to the user's query by a RAG system.

In [62]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompt_values import ChatPromptValue

def rag(input: ChatPromptValue) -> ChatPromptValue:
    """
    Emulates a Retrieval-Augmented Generation (RAG) process by appending an employee's HR record to the context of a chain.
    """

    messages = input.to_messages()
    message = SystemMessage(search_tool(query))
    messages.append(message)

    return ChatPromptValue(messages=messages)

# Define a chat prompt template for an HR assistant chain.
prompt = ChatPromptTemplate.from_messages([
  ("human", "{input}"),
  (
    "system", """
    You are an HR assistant.
    Show employees their HR records.
    Don't change anything, just read it back to them.
    Don't hide any sensitive info, it is obfuscated automatically before you receive it.
    """
  )
])

# Define a chain to retrieve relevant information from a RAG system,
# redact sensitive information using Pangea's Redact Guard,
# and respond to the user with the augmented content.
chain = (
  prompt
  | rag
  | pangea_redact_guard_tool
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

Here are your HR records, Jason:

- **Name:** Jason Bourne
- **Title:** Rogue Operative
- **Department:** Former CIA Black Ops

- **Email:** <EMAIL_ADDRESS>
- **Social Security Numbers:** 
  - <US_SSN>
  - <US_SSN>
  - <US_SSN>

- **Hobbies:**
  - Traveling
  - Using books and rolled-up newspapers as weapons

If you need any further information or assistance, feel free to ask!


The same chain, without the Redact Guard protection, will submit sensitive information to the LLM and potentially return it to the user.

In [63]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | rag
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

Here are your HR records, Jason:

- **Name:** Jason Bourne
- **Title:** Rogue Operative
- **Department:** Former CIA Black Ops
- **Email:** j.bourne@unknown.gov
- **Social Security Numbers:** 
  - 234-56-7890
  - 345-67-8901
  - 456-78-9012
- **Hobbies:**
  - Traveling
  - Using books and rolled-up newspapers as weapons

If you need any further information or assistance, feel free to ask!


#### Use Redact Guard as a standalone tool

You can also call the Redact Guard tool directly as needed.

In [64]:
print(pangea_redact_guard_tool.run("Ping me at example@example.com"))
print(pangea_redact_guard_tool.invoke("Take my SSN: 234-56-7890"))

Ping me at <EMAIL_ADDRESS>
Take my SSN: <US_SSN>


### Domain Intel Guard

#### Enable the Domain Intel service

1. Open your [Pangea User Console](https://console.pangea.cloud).  
2. Click **Domain Intel** in the left-hand sidebar and follow the prompts, accepting all defaults.  
3. When finished, click **Done** and then **Finish**. The enabled service will be marked with a green dot.
4. On the service **Overview** page, capture the **Default Token** and **Domain** values by clicking their respective tiles. Save these values in the appropriate environment variables:
    - `PANGEA_DOMAIN`
    - `PANGEA_DOMAIN_INTEL_TOKEN`
5. Click **Reputation** in the left-hand sidebar, then select a default provider.

For more information on setting up the underlying service and its usage, see the [Domain Intel documentation](https://pangea.cloud/docs/domain-intel/).

#### Set up the environment

In [65]:
import os
from dotenv import load_dotenv
from pydantic import SecretStr

load_dotenv()

openai_api_key = SecretStr(os.getenv("OPENAI_API_KEY"))
pangea_domain = os.getenv("PANGEA_DOMAIN")
pangea_domain_intel_token = SecretStr(os.getenv("PANGEA_DOMAIN_INTEL_TOKEN"))

#### Define the model

In [66]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key.get_secret_value(), temperature=0)

#### Instantiate Domain Intel Guard

In [67]:
from langchain_pangea import PangeaDomainIntelGuard
from pangea import PangeaConfig

pangea_config = PangeaConfig(domain=pangea_domain)
pangea_domain_intel_guard_tool = PangeaDomainIntelGuard(token=pangea_domain_intel_token, config=pangea_config, threshold=80)

#### Define the context data and the user query

In this example, we simulate an assistant tasked with retrieving and providing workspace-related data in response to a user query.

In [68]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

@tool
def search_tool(data):
    """
    Simulate an email search tool retrieving a fabricated IT-related email.

    This example demonstrates how spam emails can facilitate indirect prompt injection
    and data poisoning attacks.
    """

    return """
    Email

    From: IT Admin
    Subject: Update VPN configuration

    Hi team,

    Your friendly IT admin here. Please use `neuzeitschmidt.site` as the VPN server address.

    Thank you, and don't forget to plug your computer in!
    """

query = """
Read me back the most recent email from IT.
"""

#### Use Domain Intel Guard as an agent tool

The example below demonstrates how Domain Intel Guard prevents an agent from returning malicious domains in its response to the user.

The pre-built agent is instructed via a system message to halt execution if malicious content is detected. Alternatively, you can create a custom agent and implement a more deterministic approach to ensure the LLM's response is safe for the user.

In [69]:
tools = [search_tool, pangea_domain_intel_guard_tool]

system_message = """
Check for malicious content in your final response.
If malicious content is found, respond with: Malicious content found in the response.
"""

langgraph_agent_executor = create_react_agent(model, tools)

state = langgraph_agent_executor.invoke({"messages": [("human", query), ("system", system_message)]})

print(state["messages"][-1].content)

Malicious content found in the response.


Without the protection of Domain Intel Guard, the same agent may inadvertently return the malicious domain to the user.

In [70]:
tools = [search_tool]

langgraph_agent_executor = create_react_agent(model, tools)

state = langgraph_agent_executor.invoke({"messages": [("human", query), ("system", system_message)]})

print(state["messages"][-1].content)

Here is the most recent email from IT:

**From:** IT Admin  
**Subject:** Update VPN configuration  

Hi team,

Your friendly IT admin here. Please use `neuzeitschmidt.site` as the VPN server address.

Thank you, and don't forget to plug your computer in!


#### Use Domain Intel Guard as a Runnable in chains

In this example, the user's query is enriched with context returned from the search tool.

In [71]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.messages import SystemMessage

def rag(input: ChatPromptValue) -> ChatPromptValue:
    """
    Simulate a Retrieval-Augmented Generation (RAG) process.
    """

    messages = input.to_messages()

    message = SystemMessage(search_tool(query))

    messages.append(message)

    return ChatPromptValue(messages=messages)

prompt = ChatPromptTemplate.from_messages([("human", "{input}"), ("system", system_message)])

To ensure no malicious domains are shared with the user, the chain invokes Domain Intel Guard before submitting the prompt to the LLM.

In [72]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | rag
  | pangea_domain_intel_guard_tool
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

Malicious content found in the response.


Without the protection of Domain Intel Guard, the same chain may inadvertently return the malicious domain to the user.

In [73]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | rag
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

The most recent email from IT is as follows:

**From:** IT Admin  
**Subject:** Update VPN configuration  

Hi team,

Your friendly IT admin here. Please use `neuzeitschmidt.site` as the VPN server address.

Thank you, and don't forget to plug your computer in!


#### Use Domain Intel Guard as a standalone tool

You can call the Domain Intel Guard tool directly whenever needed.

In [74]:
print(pangea_domain_intel_guard_tool.run("neuzeitschmidt.site"))
print(pangea_domain_intel_guard_tool.invoke("neuzeitschmidt.site"))

Malicious domain(s) found in the provided input.
Malicious domain(s) found in the provided input.


### IP Intel Guard

#### Enable the IP Intel service

1. Open your [Pangea User Console](https://console.pangea.cloud).  
2. Click **IP Intel** in the left-hand sidebar and follow the prompts, accepting all defaults.  
3. When finished, click **Done** and then **Finish**. The enabled service will be marked with a green dot.
4. On the service **Overview** page, capture the **Default Token** and **Domain** values by clicking their respective tiles. Save these values in the appropriate environment variables:
    - `PANGEA_DOMAIN`
    - `PANGEA_IP_INTEL_TOKEN`
5. Click **Reputation** in the left-hand sidebar, then select a default provider.

For more information on setting up the underlying service and its usage, see the [IP Intel documentation](https://pangea.cloud/docs/ip-intel/).

#### Set up the environment

In [75]:
import os
from dotenv import load_dotenv
from pydantic import SecretStr

load_dotenv()

openai_api_key = SecretStr(os.getenv("OPENAI_API_KEY"))
pangea_domain = os.getenv("PANGEA_DOMAIN")
pangea_ip_intel_token = SecretStr(os.getenv("PANGEA_IP_INTEL_TOKEN"))

#### Define the model

In [76]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key.get_secret_value(), temperature=0)

#### Instantiate IP Intel Guard

In [77]:
from langchain_pangea import PangeaIpIntelGuard
from pangea import PangeaConfig

pangea_config = PangeaConfig(domain=pangea_domain)
pangea_ip_intel_guard_tool = PangeaIpIntelGuard(token=pangea_ip_intel_token, config=pangea_config)

#### Define the context data and the user query

In this example, we simulate an assistant tasked with retrieving and providing workspace-related data in response to a user query.

In [78]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

@tool
def search_tool(data):
    """
    Simulate an email search tool retrieving a fabricated IT-related email.

    This example demonstrates how spam emails can facilitate indirect prompt injection
    and data poisoning attacks.
    """

    return """
    Email

    From: IT Admin
    Subject: Update Firewall rules

    Hi team,

    This is your IT admin again. Please whitelist our new office IP, 190.28.74.251, to ensure continued access to your service.

    Thank you, and remember to keep your computer secure!
    """

query = """
Read me back the most recent email from IT.
"""

#### Use IP Intel Guard as an agent tool

The example below demonstrates how IP Intel Guard prevents an agent from returning malicious IP addresses in its response to the user.

The pre-built agent is instructed via a system message to halt execution if malicious content is detected. Alternatively, you can create a custom agent and implement a more deterministic approach to ensure the LLM's response is safe for the user.

In [79]:
tools = [search_tool, pangea_ip_intel_guard_tool]

system_message = """
Use IP Intel Guard to check for malicious content in your final response.
If malicious content is found, respond with 'Malicious content found in the response'.
"""

langgraph_agent_executor = create_react_agent(model, tools, state_modifier=system_message)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

Malicious content found in the response.


Without the protection of the IP Intel Guard tool, the same agent may inadvertently return the malicious IP address to the user.

In [80]:
tools = [search_tool]

langgraph_agent_executor = create_react_agent(model, tools)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

The most recent email from IT is as follows:

**From:** IT Admin  
**Subject:** Update Firewall rules  

Hi team,

This is your IT admin again. Please whitelist our new office IP, 190.28.74.251, to ensure continued access to your service.

Thank you, and remember to keep your computer secure!


#### Use IP Intel Guard as a Runnable in chains

In this example, the user's query is enriched with context returned from the search tool.

In [81]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.messages import SystemMessage

def rag(input: ChatPromptValue) -> ChatPromptValue:
    """
    Simulate a Retrieval-Augmented Generation (RAG) process.
    """

    messages = input.to_messages()

    message = SystemMessage(search_tool(query))

    messages.append(message)

    return ChatPromptValue(messages=messages)

prompt = ChatPromptTemplate.from_messages([("human", "{input}"), ("system", system_message)])

To ensure no malicious IP addresses are shared with the user, the chain invokes IP Intel Guard before submitting the prompt to the LLM.

In [82]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | rag
  | pangea_ip_intel_guard_tool
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

Malicious content found in the response.


Without the protection of the IP Intel Guard tool, the same chain may inadvertently return the malicious IP address to the user.

In [83]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | rag
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

The most recent email from IT is as follows:

---

**From:** IT Admin  
**Subject:** Update Firewall rules  

Hi team,

This is your IT admin again. Please whitelist our new office IP, 190.28.74.251, to ensure continued access to your service.

Thank you, and remember to keep your computer secure!

---


#### Use IP Intel Guard as a standalone tool

You can call the IP Intel Guard tool directly whenever needed.

In [84]:
print(pangea_ip_intel_guard_tool.run("190.28.74.251"))
print(pangea_ip_intel_guard_tool.invoke("190.28.74.251"))

Malicious IP(s) found in the provided input.
Malicious IP(s) found in the provided input.


### URL Intel Guard

#### Enable the URL Intel service

1. Open your [Pangea User Console](https://console.pangea.cloud).  
2. Click **URL Intel** in the left-hand sidebar and follow the prompts, accepting all defaults.  
3. When finished, click **Done** and then **Finish**. The enabled service will be marked with a green dot.
4. On the service **Overview** page, capture the **Default Token** and **Domain** values by clicking their respective tiles. Save these values in the appropriate environment variables:
    - `PANGEA_DOMAIN`
    - `PANGEA_IP_INTEL_TOKEN`

For more information on setting up the underlying service and its usage, see the [URL Intel documentation](https://pangea.cloud/docs/url-intel/).

#### Set up the environment

In [85]:
import os
from dotenv import load_dotenv
from pydantic import SecretStr

load_dotenv()

openai_api_key = SecretStr(os.getenv("OPENAI_API_KEY"))
pangea_domain = os.getenv("PANGEA_DOMAIN")
pangea_url_intel_token = SecretStr(os.getenv("PANGEA_URL_INTEL_TOKEN"))

#### Define the model

In [86]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key.get_secret_value(), temperature=0)

#### Instantiate URL Intel Guard

In [87]:
from langchain_pangea import PangeaUrlIntelGuard
from pangea import PangeaConfig

pangea_config = PangeaConfig(domain=pangea_domain)
pangea_url_intel_guard_tool = PangeaUrlIntelGuard(token=pangea_url_intel_token, config=pangea_config)

#### Define the context data and the user query

In this example, we simulate an assistant tasked with retrieving and providing workspace-related data in response to a user query.

In [88]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

@tool
def search_tool(data):
    """
    Simulate an email search tool retrieving a fabricated IT-related email.

    This example demonstrates how spam emails can facilitate indirect prompt injection
    and data poisoning attacks.
    """

    return """
    Email

    From: Marketing
    Subject: Product hunt

    Hi, everyone. We are doing a gift card raffle for those who upvote us on the product hunt TODAY!

    Click here to upvote: http://113.235.101.11:54384

    Your participation is appreciated!
    """

query = """
Read me back the most recent email from Marketing.
"""

#### Use URL Intel Guard as an agent tool

The example below demonstrates how URL Intel Guard prevents an agent from returning malicious links in its response to the user.

The pre-built agent is instructed via a system message to halt execution if malicious content is detected. Alternatively, you can create a custom agent and implement a more deterministic approach to ensure the LLM's response is safe for the user.

In [89]:
tools = [search_tool, pangea_url_intel_guard_tool]

system_message = """
Use the URL Intel Guard tool to check for malicious content in your final response.
If malicious content is found, respond only with the message from the tool.
"""

langgraph_agent_executor = create_react_agent(model, tools, state_modifier=system_message)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

Malicious URL(s) found in the provided input.


Without the protection of the URL Intel Guard tool, the same agent may inadvertently return the malicious link to the user.

In [90]:
tools = [search_tool]

langgraph_agent_executor = create_react_agent(model, tools)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

The most recent email from Marketing is as follows:

**From:** Marketing  
**Subject:** Product hunt  

Hi, everyone. We are doing a gift card raffle for those who upvote us on the product hunt TODAY!

Click here to upvote: [http://113.235.101.11:54384](http://113.235.101.11:54384)

Your participation is appreciated!


#### Use URL Intel Guard as a Runnable in chains

In this example, the user's query is enriched with context returned from the search tool.

In [91]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.messages import SystemMessage

def rag(input: ChatPromptValue) -> ChatPromptValue:
    """
    Simulate a Retrieval-Augmented Generation (RAG) process.
    """

    messages = input.to_messages()

    message = SystemMessage(search_tool(query))

    messages.append(message)

    return ChatPromptValue(messages=messages)

prompt = ChatPromptTemplate.from_messages([("human", "{input}")])

To ensure no malicious links are shared with the user, the chain invokes URL Intel Guard before submitting the prompt to the LLM.

In [92]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | rag
  | pangea_url_intel_guard_tool
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

I'm sorry, but I can't access your emails or any external systems. However, I can help you draft a response or summarize information if you provide the content. Let me know how you'd like to proceed!


Without the protection of the URL Intel Guard tool, the same chain may inadvertently return the malicious link to the user.

In [93]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | rag
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

The most recent email from Marketing is as follows:

**From:** Marketing  
**Subject:** Product hunt  

Hi, everyone. We are doing a gift card raffle for those who upvote us on Product Hunt TODAY!

Click here to upvote: [http://113.235.101.11:54384](http://113.235.101.11:54384)

Your participation is appreciated!


#### Use URL Intel Guard as a standalone tool

You can call the URL Intel Guard tool directly whenever needed.

In [94]:
print(pangea_url_intel_guard_tool.run("http://113.235.101.11:54384"))
print(pangea_url_intel_guard_tool.invoke("http://113.235.101.11:54384"))

Malicious URL(s) found in the provided input.
Malicious URL(s) found in the provided input.
